In [122]:
import pandas as pd
import numpy as np
import re
import nltk
import emoji
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [28]:
import tweepy

BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAM8j0AEAAAAA0U89kyhF1RpEJd1tFb3SaL800wo%3D047zPSpIojMHSZ32V5X6gKtLooptdoZ1dONUfnmM4sbggDjufB"

client = tweepy.Client(bearer_token=BEARER_TOKEN)

In [29]:
keywords = ["stock market", "finance", "economy"]
Query = " OR ".join(keywords) + " -is:retweet"

all_tweets = []


response = client.search_recent_tweets(
    query=Query,
    max_results=20, 
    tweet_fields=["created_at", "author_id"]
)
if response.data:
    for tweet in response.data:
        all_tweets.append({
            "text": tweet.text, 
            "created_at": tweet.created_at
        })

In [41]:
for tweet in all_tweets:
    print(tweet)

{'text': "The future of sports finance is here Investors will soon be able to buy into their favorite teams' ownership stakes", 'created_at': datetime.datetime(2025, 3, 19, 11, 7, 11, tzinfo=datetime.timezone.utc)}
{'text': "RT @YourAnonNews: As the stock market continues it's tumble there are a few MAGA who are voicing that they'd like a refund.\nNo. Fuck that.…", 'created_at': datetime.datetime(2025, 3, 19, 11, 7, 11, tzinfo=datetime.timezone.utc)}
{'text': '@chauffeur243 Il finance seulement les journalistes et communicateurs pour sa visibilité \n🤣🤣🤣🤣', 'created_at': datetime.datetime(2025, 3, 19, 11, 7, 11, tzinfo=datetime.timezone.utc)}
{'text': 'RT @nirvana_io: The AI-Native Financial Paradigm Shift\n\nTraditional finance was designed for human decision-makers, but the future belongs…', 'created_at': datetime.datetime(2025, 3, 19, 11, 7, 11, tzinfo=datetime.timezone.utc)}
{'text': 'Game-changer in the world of finance is the emergence of yield farming, where investors are rewarde

In [38]:
tweets_list = all_tweets.copy()

In [69]:
pd.DataFrame(all_tweets)

,text,created_at
0,The future of sports finance is here Investors...,2025-03-19 11:07:11+00:00
1,RT @YourAnonNews: As the stock market continue...,2025-03-19 11:07:11+00:00
2,@chauffeur243 Il finance seulement les journal...,2025-03-19 11:07:11+00:00
3,RT @nirvana_io: The AI-Native Financial Paradi...,2025-03-19 11:07:11+00:00
4,Game-changer in the world of finance is the em...,2025-03-19 11:07:11+00:00
5,RT @TheBonkKong: 🦍🦧🔥Launch 1 hour after pre-sa...,2025-03-19 11:07:11+00:00
6,@Egmbsc Regarding #IntelligentFuture and the i...,2025-03-19 11:07:11+00:00
7,Mozambique’s textile industry recorded slight ...,2025-03-19 11:07:10+00:00
8,RT @nirvana_io: The AI-Native Financial Paradi...,2025-03-19 11:07:08+00:00
9,RT @LauraLoomer: JUST IN:\n\n⁦@JDVance⁩ will s...,2025-03-19 11:07:08+00:00


In [68]:
nlp = spacy.load("en_core_web_sm")

def preprocess_tweets(tweet):
    tweet = emoji.replace_emoji(tweet,replace="")
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE) 
    tweet = re.sub(r'\@\w+|\#', '', tweet)
    tweet = re.sub(r'[^\w\s]', '', tweet)
    tweet = tweet.lower()
    doc = nlp(tweet)
    words = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]

    return " ".join(words)

cleaned_tweets = [preprocess_tweets(tweet['text']) for tweet in all_tweets]

In [123]:
pd.DataFrame(cleaned_tweets)

,0
0,future sport finance investor soon able buy fa...
1,rt stock market continue tumble maga voice d l...
2,il finance seulement les journaliste et commun...
3,rt ainative financial paradigm shift tradition...
4,gamechanger world finance emergence yield farm...
5,rt launch hour presale buy
6,intelligentfuture impact smart protocol global...
7,mozambiques textile industry record slight gro...
8,rt ainative financial paradigm shift tradition...
9,rt serve rnc finance chairman time history republ


In [124]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(cleaned_tweets)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(cleaned_tweets)
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')

In [126]:
print(cleaned_tweets)

['future sport finance investor soon able buy favorite team ownership stake', 'rt stock market continue tumble maga voice d like refund fuck', 'il finance seulement les journaliste et communicateur pour sa visibilité', 'rt ainative financial paradigm shift traditional finance design human decisionmaker future belong', 'gamechanger world finance emergence yield farming investor reward provide liquidity lend protocol earn', 'rt launch hour presale buy', 'intelligentfuture impact smart protocol global economy foresee shift efficient resource allocation dynamic pricing model', 'mozambiques textile industry record slight growth production value m m mzn accord official budget execution data africa mozambique news economy', 'rt ainative financial paradigm shift traditional finance design human decisionmaker future belong', 'rt serve rnc finance chairman time history republ', 'win not luck calculated risk strategic invest future finance number meet intuition', 'rt dude million bam stock option

In [127]:
embedding_index = {}
with open("glove.6B.100d.txt", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        embedding_index[word] = np.array(values[1:], dtype=np.float32)

embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [128]:
print(embedding_matrix.shape)  # Should be (len(word_index) + 1, 100)


(173, 100)


In [129]:
model = Sequential([
    Embedding(input_dim=len(word_index) + 1, output_dim=100, weights=[embedding_matrix], input_length=100, trainable=True),
    Bidirectional(LSTM(128, return_sequences=True)),  
    Dropout(0.3),
    Bidirectional(LSTM(64)),  
    Dropout(0.3),
    Dense(64, activation="relu"),
    Dense(3, activation="softmax")  
])

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

C:\Users\Prajwal\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [130]:
model.build(input_shape=(None, 100))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 100)       │        17,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 100, 256)       │       234,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 424,599 (1.62 MB)

 Trainable params: 424,599 (1.62 MB)

 Non-trainable params: 0 (0.00 B)

In [131]:
negative_news = ["loss", "crash", "decline", "drop", "sell-off", "resigns", "fraud", "lawsuit", "regulatory probe"]
positive_news = ["profit", "growth", "record high", "buy", "strong earnings", "acquisition", "surge", "rally"]
neutral_news = ["mixed market", "announcement", "economic data", "unchanged", "expected results"]
speculative_news = ["rumor", "speculation", "potential", "unconfirmed", "analyst predicts", "expected to"]

labels = [
    0 if any(word in tweet.lower() for word in negative_news) else 
    1 if any(word in tweet.lower() for word in positive_news) else 
    2 if any(word in tweet.lower() for word in neutral_news) else 
    2 
    for tweet in cleaned_tweets
]  
labels = to_categorical(labels, num_classes=3)

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)


In [132]:
padded_sequences.shape

(19, 100)

In [133]:
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(15, 100) (15, 3)
(4, 100) (4, 3)


In [134]:
from tensorflow.keras import backend as K
K.clear_session()

In [135]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.0667 - loss: 1.1311 - val_accuracy: 0.5000 - val_loss: 1.0392
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step - accuracy: 0.8667 - loss: 0.9896 - val_accuracy: 0.5000 - val_loss: 1.0057
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step - accuracy: 0.9333 - loss: 0.8601 - val_accuracy: 0.5000 - val_loss: 0.9861
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - accuracy: 0.9333 - loss: 0.7162 - val_accuracy: 0.5000 - val_loss: 1.0139
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - accuracy: 0.9333 - loss: 0.5211 - val_accuracy: 0.5000 - val_loss: 1.1041
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - accuracy: 0.9333 - loss: 0.3690 - val_accuracy: 0.5000 - val_loss: 1.3017
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step - accuracy: 0.9333 - loss: 0.3279 - val_accuracy: 0.5000 - val_loss: 1.6213
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step - accuracy: 0.9333 - loss: 0.2887 - val_accuracy: 0.5000 - val_loss: 1.

In [136]:
model.save("twitter_sentiment_model.h5")

In [139]:
prediction = model.predict(padded_sequences)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step


In [140]:
prediction.shape

(19, 3)

In [142]:
labels = ["negative_news","positive_news","neutral_news"]

predicted_indices=np.argmax(prediction,axis=1)

sentiments = [labels[i] for i in predicted_indices]
print(sentiments)

['neutral_news', 'neutral_news', 'neutral_news', 'neutral_news', 'neutral_news', 'neutral_news', 'neutral_news', 'neutral_news', 'neutral_news', 'neutral_news', 'neutral_news', 'neutral_news', 'neutral_news', 'neutral_news', 'neutral_news', 'neutral_news', 'neutral_news', 'neutral_news', 'neutral_news']
